In [1]:
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Books',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'Tshirt.csv']

In [2]:
YEARS_TO_GET = [2000,2001]

**Start OUTER Loop**

Set up a progress bar - we want to keep track of our progress and ensure our calls are working. The progress bar works within the for statement of the loop. Note that this will iterate through each year that is defined in the YEARS_TO_GET variable. 

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):

Ultimately we will be creating a loop, but let's explore each piece of the code. 

**Select a JSON_FILE filename to save the results in progress**

Check if the file exists
if not: create an empty JSON file with *with open* that just contains the key "imdb_id" 
if yes: Do nothing

First, define the file path and names: We are going to have multiple files since we are creating a separate file for each year. The code below will identify the folder in the FOLDER we just defined above and will name the file based on the current year. 

In [ ]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

#Check if that file already exists or not

# Check if file exists
file_exists = os.path.isfile(JSON_FILE)

In [ ]:
# If it does not exist: create it
if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

**Define/Filter the IDs to Call**

You need to read in the filtered dataframe you created based on the specification of Project 3 Part 1. 

In [ ]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('YOUR PATH')

We are going to break up data by year, so we will define a new dataframe for each year. Notice that which YEAR will depend on what we define YEAR as. Leaving YEAR a variable allows the code to be easier to read and reproduce. 

In [ ]:
#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()#.to_list()

**Check for and Remove any Previously Downloaded Movie IDs**

In [ ]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

**Check for and filter out movie IDs that already exist**

This next line will prevent you from wasting API calls on data you already have. Note that it is defining the IDs you are calling in such a way that it excludes any IDs that are already present in the previous_df. 

In [ ]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

Now, we have defined the 'movie_ids_to_get'. It includes the IDs from our dataframe in the year we are seeking, and it excludes any that we have already made calls for. 

**Start INNER Loop**

Use a function to append new results to the existing JSON file

In [3]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

**Iterate through the list of Movie IDs and make the calls**


In [5]:
#Get index and movie id from list
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue

NameError: name 'tqdm_notebook' is not defined

Save the years results as csv.gz file

In [ ]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

If you get an error message, try replacing pd.read_json with pd.read_and_fix_json

In [ ]:
# Instead of previous_df=pd.read_json:
previous_df = read_and_fix_json(JSON_FILE)